# 20 · Network → Timeline (Synthetic PCAP/JSON Fallback)

We avoid shipping binary PCAPs. Instead we synthesize a minimal capture or feed JSON to the network module.

In [ ]:
# Generate a tiny synthetic 'pcap-like' JSON for analysis.
from pathlib import Path
import json

lab = Path.cwd() / "lab_workspace"
case_id = "lab_demo_case"
case_dir = lab / "cases" / case_id
analysis_dir = case_dir / "analysis" / "network"
analysis_dir.mkdir(parents=True, exist_ok=True)

synthetic = {
    "packets": [
        {"ts":"2025-01-01T00:00:01Z","src":"10.0.0.5","dst":"1.2.3.4","proto":"TCP","dport":443},
        {"ts":"2025-01-01T00:00:03Z","src":"10.0.0.5","dst":"192.168.1.100","proto":"TCP","dport":4444},
        {"ts":"2025-01-01T00:00:05Z","src":"10.0.0.5","dst":"8.8.8.8","proto":"UDP","dport":53,"dns_query":"example.com"}
    ]
}
json_path = analysis_dir / "synthetic_network.json"
json_path.write_text(json.dumps(synthetic, indent=2, sort_keys=True))
json_path

In [ ]:
# Invoke the SDK network module with JSON input (guarded path).
from forensic.modules.analysis.network import NetworkAnalysisModule
from forensic.core.framework import ForensicFramework

fw = ForensicFramework(workspace=Path.cwd() / "lab_workspace")
case = fw.load_case("lab_demo_case")
mod = NetworkAnalysisModule()

params = {"pcap_json": str(json_path), "dry_run": False}

# Framework may provide execute_module; if not, call module directly
try:
    res = fw.execute_module("network", params=params)
except Exception:
    res = mod.run(fw, case, params)

print(type(res))
try:
    import json as _json
    print(_json.dumps(res.get("summary", {}), indent=2, sort_keys=True)[:800])
except Exception as e:
    print("Summary not available:", e)

### Build a minimal timeline CSV from the synthesized flows

In [ ]:
# Produce a minimal CSV timeline based on synthetic events.
import csv
from pathlib import Path

timeline_csv = (Path.cwd() / "lab_workspace" / "cases" / "lab_demo_case" / "analysis" / "network" / "synthetic_timeline.csv")
timeline_csv.parent.mkdir(parents=True, exist_ok=True)
rows = [
    ("2025-01-01T00:00:01Z","net","TCP 10.0.0.5 → 1.2.3.4:443"),
    ("2025-01-01T00:00:03Z","net","TCP 10.0.0.5 → 192.168.1.100:4444 (suspicious)"),
    ("2025-01-01T00:00:05Z","dns","Query example.com via 8.8.8.8")
]
with timeline_csv.open("w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["timestamp","category","detail"])
    for r in rows:
        w.writerow(r)
(str(timeline_csv), timeline_csv.read_text().splitlines()[:4])